In [7]:
import numpy as np
import math

In [17]:
class Scalar:

    def __init__(self, data, _children = (), _op = ''):
        self.data = data
        self._prev = set(_children)
        self.grad = 0.0
        self._backward = lambda : None

    def __repr__(self):
        return f"Scalar(data: {self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward
        return out

    def __radd__(self, other):
        return self + other
    
    def __mul__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other

    def __neg__(self):
        return (self * -1)

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Only supporting int and float powers"
        out = Scalar(self.data ** other)

        def _backward():
            self.grad += (other * (self.data ** (other - 1))) * out.grad

        out._backward = _backward
        return out

    def relu(self):
        d = self.data

        def _backward():
            self.grad += (d > 0) * out.grad
        
        out._backward = _backward
        out = Scalar(d * (d > 0))
        return out

    def backward(self):
        self.grad = 1.0
        self._backward()

In [27]:
class Neuron:

    def __init__(self, nin):
        self.w = [Scalar(np.random.uniform(-1,1)) for _ in range(nin)]
        self.b = Scalar(np.random.uniform(-1, 1))

    def __call__(self, xin):
        out = sum(wi * xi for wi,xi in zip(self.w, xin))
        out += self.b
        return out
    
    def __repr__(self):
        return f"Weights : {self.w}\nBias : {self.b}"

    def parameters(self):
        return self.w + [self.b]

class Layer:

    def __init__(self, nin, nneurons):
        self.neurons = [Neuron(nin) for _ in range(nneurons)]

    def __call__(self, xin):
        return [n(xin) for n in self.neurons]
    
    def __repr__(self):
        return f"Neurons : {self.neurons}"

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

class MLP:

    def __init__(self, nin, llayers):
        sz = [nin] + llayers
        self.layers = [Layer(sz[i], sz[i + 1])for i in range(len(llayers))]

    def __call__(self, xin):
        for layer in self.layers:
            xin = layer(xin)
        return xin

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
    
    def __repr__(self):
        return f"Layers: \n{self.layers}"
        

In [29]:
x = [1.0, 2.0, 3.0, -2.0]

In [43]:
m = MLP(4, [3, 2, 1])

In [47]:
m(x)

[Scalar(data: 0.42581489095549285)]